In [5]:
import numpy as np
import pickle
from copy import deepcopy
import h5py
import json

In [2]:
# Build the dict which maps Geohash index to vector index
def build_dic(flights):
    flat_list = [i for j in flights for i in j]
    char = [i for j in flat_list for i in j]
    char_idx = {c: i for i, c in enumerate(sorted(set(char)))}
    return char_idx

In [2]:
# Break the Geohash seq list into sets of n points
def create_input(flights,number_points):
    input_sq = []
    output_val = []
    for i in flights:
        N = range(number_points,len(i))
        for k in N:
            # sq = []
            a = [q for p in i[k-number_points:k] for q in p]
            b = [q for p in i[k] for q in p]
            for m in range(0,6):
                q = deepcopy(a)
                q.extend(b[0:m])
                input_sq.append(q)
                output_val.append(b[m])
    return input_sq, output_val

In [2]:
# Turn Seq of N points into a set of training vectors
def create_vectors(char_idx,input_sq,output_val):
    X = np.zeros((len(input_sq), 23, len(char_idx)), dtype=bool)
    Y = np.zeros((len(output_val),len(char_idx)),dtype=bool)
    for i, seq in enumerate(input_sq):
        for t, char in enumerate(seq):
            X[i, t, char_idx[char]] = 1
        Y[i, char_idx[output_val[i]]] = 1

    return X,Y

In [3]:
# Load GeoHash strings
data = pickle.load( open( "../data/geohash_6.p", "rb" ) )

In [7]:
# Create Geohash index
char_idx = build_dic(data)

In [8]:
# Save Geohash index for later use
with open('../data/char_dict.json', 'w') as f:
    json.dump(char_idx, f)

In [16]:
# Creat the slice the training into sections
input_sq, output_val = create_input(data,3)

In [25]:
# Create X and Y vectors for training the network
X,Y = create_vectors(char_idx,input_sq,output_val)

/tmp/ipykernel_26811/2263149941.py:25: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(input_sq), 23, len(char_idx)), dtype=np.bool)
/tmp/ipykernel_26811/2263149941.py:26: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y = np.zeros((len(output_val),len(char_idx)),dtype=np.bool)


In [32]:
# Save the training data in a hdf5 file
with h5py.File("../data/training_data.hdf5", "w") as f:
    f.create_dataset("X", maxshape=(None, 23, 32), data=X)
    f.create_dataset("Y", maxshape=(None, 32), data=Y)